In [24]:
import pandas as pd
import numpy as np
import re
import sys

# np.set_printoptions(threshold=sys.maxsize)

In [25]:
inputFile = 'data_formatted.csv'
outputFile = 'data_formatted_2.csv'

In [51]:
#read
df = pd.read_csv(inputFile, sep=',')

In [27]:
def split(columnName, seperator, newName = None, dropNaN=True):
    if newName == None:
        newName = columnName
    newFrame = []
    for _, row in df.iterrows():
        _id = row['id']
        item = str(row[columnName])
        if dropNaN and item == "nan":
            continue
        pieces = item.split(seperator)
        for piece in pieces:
            newFrame.append([_id, piece.strip()])
    return pd.DataFrame(newFrame, columns=['id', newName])

In [28]:
illegalCharsRegex = '[^üéöíøóäñÜÖßáÄ¢Šç’´‐\w\'\-\s.,]'
def findIllegalNames(data, column, regex):
    copy = data[column].dropna()
    return copy[copy.str.contains(regex, regex=True, flags= re.I)]

In [48]:
df

,Unnamed: 0,Unnamed: 0.1,id,type,title,authors,editors,oldDepartmentNames,publisher,journal,...,2629,2630,2631,2633,2634,2635,2636,2637,2638,2639
0,0.0,0.0,1.0,inbook,Wer sind die Experten? User Needs Analysis (UN...,"Linneweber, V.","Harloff, H.J.",Library,Verlag für Angewandte Psychologie,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,2.0,confpaper,Impact of climate change on forests: applicati...,"Lindner, M.; Bugmann, H.; Cramer, W.; Lasch, P.","Vancura, K.; Sramek, V.",Global Change & Natural Systems,Forestry and Game Management Research Institute,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,2.0,3.0,lecture,Quantifizierung des Einflusses verschiedener (...,"Bronstert, A.",NaN,Global Change & Natural Systems,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,3.0,4.0,confpaper,Ein Modell der Sukzessionsdynamik europäischer...,"Schaber, J.; Badeck, F.-W.; Lasch, P.","Pelz, D.R.; Rau, O.; Saborowski, J.",Global Change & Natural Systems,Deutscher Verband forstlicher Versuchsanstalte...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,4.0,5.0,confpaper,Comparison of data requirements and performanc...,"Krysanova, V.; Bronstert, A.; Menzel, L.",NaN,Global Change & Natural Systems,Dt. Nationalkomitee f. d. IHP der UNESCO u. d....,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,5.0,6.0,paperr,Delta 13C of CO2 respired in the dark in relat...,"Duranceau, M.; Ghashghaie, J.; Badeck, F.-W.; ...",NaN,Global Change & Natural Systems,NaN,"Plant, Cell and Environment",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6.0,6.0,7.0,inbook,Land use change in Europe and implications for...,"Krysanova, V.; Wechsung, F.; Meiner, A.; Vasil...","Ennuste, Ü.; Wilder, L.",Global Change & Natural Systems,"Estonian Institute of Economics, Tallinn TU",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7.0,7.0,8.0,confpaper,Großräumige hydrologische Parametrisierung und...,"Bronstert, A.; Güntner, A.; Jaeger, A.; Krol, ...","Fohrer, N.; Döll, P.","Global Change & Natural Systems, Integrated Sy...",Kassel University Press,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8.0,8.0,9.0,confpaper,The use of semi-qualitative reasoning and prob...,"Moldenhauer, O.; Bruckner, T.; Petschel-Held, G.",NaN,Integrated Systems Analysis,PIK; IOS Press,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9.0,9.0,10.0,confpaper,Effects of climate change influencing storm ru...,"Bronstert, A.; Bürger, G.; Heidenreich, M.; Ka...","Casale, R.; Samuels, P.; Bronstert, A.","Global Change & Natural Systems, Climate Research",Office for Official Publications of the Europe...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
#shortens name: Lastname, F.S.
def shortenNames(columnName):
    column = df[columnName] 
    wrong = []

    for x in range(len(column)):
        end = []
        
        #checkes whether value is nan
        if column[x] != column[x]:
            continue
            
        names = column[x].split('; ')
        for i in range(len(names)):
            ns = names[i].split(', ')
            
            #ns has to have two parts: lastname, firstname
            if len(ns) == 1:
                wrong.append([x, i])
                end.append(names[i])
                continue
            firstname = ns[1]
            lastname = ns[0].replace(',', '')
           
            #checkes whether the value alreaddy has short parts
            if (re.search('.\.', firstname)):
                newFN = firstname.replace('-', '') #.replace(' ', '')
                firstnames = newFN.split(' ')
                    
                fn = ''
                for j in range(len(firstnames)):
                    if firstnames[j] == '':
                        fn += ''
                        continue
                    if (re.search('.\.', firstnames[j])):
                        fn += firstnames[j]
                    else:
                        #first letter of name
                        fn += firstnames[j][0] + '.'
                end.append(lastname + ', ' + fn)
            else:
                newFN = firstname.replace('-', ' ') #.replace(' ', '')
                firstnames = newFN.split(' ')
                fn = ''
                for j in range(len(firstnames)):
                    if firstnames[j] == '':
                        fn += ''
                        continue
                    if len(firstnames[j]) == 1:
                        fn += firstnames[j] + '.'
                    else:
                        #first letter of name
                        fn += firstnames[j][0] + '.'
                end.append(lastname + ', ' + fn)
        df.loc[columnName, x] = '; '.join(end)

In [47]:
shortenNames('authors')

0
Linneweber, V.
1
Lindner, M.; Bugmann, H.; Cramer, W.; Lasch, P.
2
Bronstert, A.
3
Schaber, J.; Badeck, F.W.; Lasch, P.
4
Krysanova, V.; Bronstert, A.; Menzel, L.
5
Duranceau, M.; Ghashghaie, J.; Badeck, F.W.; Deleens, E.C.
6
Krysanova, V.; Wechsung, F.; Meiner, A.; Vasiljev, A.
7
Bronstert, A.; Güntner, A.; Jaeger, A.; Krol, M.; Krywkow, J.
8
Moldenhauer, O.; Bruckner, T.; Petschel-Held, G.
9
Bronstert, A.; Bürger, G.; Heidenreich, M.; Katzenmaier, D.K.
10
Bronstert, A.; Ghazi, A.; Hladny, J.; Kundzewicz, Z.
11
Sprinz, D.
12
Gräfe, J.; Poschenrieder, W.; Wechsung, F.; Kartschall, T.
13
Lahmer, W.; Becker, A.
14
Lahmer, W.
15
Lahmer, W.; Becker, A.
16
Lahmer, W.; Klöcking, B.; Pfützner, B.
17
Schellnhuber, H.J.; Enke, W.; Flechsig, M.
18
Krysanova, V.; Gerten, D.; Klöcking, B.; Becker, A.
19
Cramer, W.; Kicklighter, D.W.; Bondeau, A.; Moore, B.; Nemry, B.; Ruimy, A.; Schloss, A.
20
Cramer, W.
21
Bronstert, A.; Kundzewicz, Z.W.; Menzel, L.
22
Müller-Wohlfeil, D.I.; Lahmer, W.; Krysano

316
Becker, A.
317
Becker, A.
318
Constantin, J.; Herbst, M.H.; Kluge, W.; Morgenstern, M.; Thamm, F.; Schmidt, J.; Müller-Wohlfeil, D.I.; Wegehenkel, M.; Widmoser, P.
319
Constantin, J.; Herbst, M.H.; Kluge, W.; Morgenstern, M.; Thamm, F.; Schmidt, J.; Müller-Wohlfeil, D.I.; Wegehenkel, M.; Widmoser, P.
320
Bugmann, H.; Grote, R.; Lasch, P.; Lindner, M.; Suckow, F.
321
Svirezhev, Y.M.
322
Gerstengarbe, F.W.; Werner, P.C.
323
Helm, C.
324
Lüdeke, M.K.B.; Petschel-Held, G.
325
Schellnhuber, H.J.
326
Svirezhev, Y.
327
Schellnhuber, H.J.; Fuentes, U.
328
Woodward, F.I.; Cramer, W.
329
Lindner, M.
330
Svirezhev, Y.M.
331
Svirezhev, Y.M.
332
Leimbach, M.
333
Nakicenovic, N.; Nordhaus, W.D.; Richels, R.; Toth, F.L.
334
Krol, M.; Alcamo, J.; Leemans, R.
335
Kohlmaier, G.H.; Badeck, F.W.; Otto, R.D.; Häger, Ch.D.; Kindermann, J.; Würth, G.; Lang, T.; Jäckel, U.; Nadler, A.; Klaudius, A.; Ramge, P.; Habermehl, S.; Lüdeke, M.K.B.
336
Griffiths, R.B.; Schellnhuber, H.J.; Urbschat, H.
337
Lütkemei

690
Svirezhev, Y.M.
691
Claussen, M.; Cramer, W.
692
Wenzel, V.
693
Ulbrich, U.; Bürger, G.; Schriewer, D.; v. Storch, H.; Schmitz, G.
694
Svirezhev, Y.M.; Brovkin, V.A.; Denisenko, E.A.
696
Krysanova, V.; Bronstert, A.; Menzel, L.
697
Geisel, T.; Ketzmerick, R.; Petschel, G.
698
Österle, H.; Enke, W.
699
Prentice, I.C.; Sykes, M.T.; Cramer, W.
700
Linneweber, V.
701
Linneweber, V.
702
Linneweber, V.
703
Cramer, W.
704
Linneweber, V.
705
Lasch, P.; Lindner, M.
706
Lindner, M.
707
Lindner, M.; Lasch, P.; Cramer, W.
708
Solomon, A.M.; Prentice, I.C.; Leemans, R.; Cramer, W.
709
Koelle, B.
710
Lindner, M.
711
Jonas, M.; Ganopolski, A.V.; Krabec, J.; Olendrzynski, K.P.
712
Toth, F.L.
713
Smit, B.; Burton, I.; Klein, R.J.T.; Wandel, J.
714
Petschel-Held, G.
715
Brovkin, V.; Claussen, M.; Petoukhov, V.; Ganopolski, A.
716
Toth, F.
717
Helm, C.
718
Becker, A.; Krysanova, V.; Müller-Wohlfeil, D.I.; Lahmer, W.
719
Helm, C.
720
Linneweber, V.
721
Helm, C.
722
Enke, W.
723
Lasch, P.; Lindner, M.


938
Werner, P.C.; Gerstengarbe, F.W.; Ebeling, W.
939
Ganopolski, A.V.; Jonas, M.; Krabec, J.; Olendrzynski, K.P.; Venevsky, S.
940
Bronstert, A.; Kundzewicz, Z.W.; Menzel, L.
941
Schellnhuber, H.J.; Sprinz, D.
942
Schuster, C.; Wechsung, F.; Diepenbrock, W.
943
Bruckner, T.; Petschel-Held, G.; Tóth, F.L.; Füssel, H.M.; Helm, C.; Leimbach, M.; Schellnhuber, H.J.
944
Leimbach, M.
945
Klein, R.J.T.
946
Fosberg, M.A.
947
Böhm, U.
948
Toth, F.
949
Bronstert, A.; Krysanova, V.; Schröder, A.; Becker, A.; Bork, H.
950
Gerstengarbe, F.W.; Werner, P.C.; Böhm, U.
951
Krysanova, V.; Wechsung, F.
952
Schellnhuber, H.J.
953
Claussen, M.; Brovkin, V.; Ganopolski, A.; Kubatzki, C.; Petoukhov, V.
954
Reusswig, F.
955
Böhm, U.
956
Reusswig, F.
957
Krysanova, V.; Müller-Wohlfeil, D.I.; Becker, A.
958
Reusswig, F.
959
Ganopolski, A.
960
Helm, C.
961
Lahmer, W.
962
Toth, F.L.; Petschel-Held, G.; Bruckner, T.
963
Franck, S.; Bounama, Ch.
964
Schellnhuber, H.J.; von Bloh, W.
965
Graßl, H.; Schellnhuber, H.J

1191
Bellmann, K.; Erhard, M.; Flechsig, M.; Grote, R.; Suckow, F.
1192
Klein, R.J.T.; Nicholls, R.J.; Mimura, N.
1193
Sprinz, D.
1194
Heimbach, P.; Bauer, E.; Hasselmann, S.; Hasselmann, K.
1195
Lasch, P.
1196
Kuhlbrodt, T.
1197
Schellnhuber, H.J.; Pilardeaux, B.
1198
Claussen, M.
1199
Lahmer, W.; Becker, A.
1200
Lahmer, W.; Becker, A.
1201
Lahmer, W.; Becker, A.
1202
Lüdeke, M.K.B.; Reusswig, F.; Petschel-Held, G.
1203
Petschel-Held, G.; Lüdeke, M.K.B.; Reusswig, F.
1204
Lasch, P.; Lindner, M.; Erhard, M.; Wenzel, A.
1207
Reusswig, F.
1208
Reusswig, F.
1209
Becker, A.; Lahmer, W.
1210
Petschel-Held, G.
1211
Cassel-Gintz, M.
1212
Becker, A.; Güntner, A.; Katzenmaier, D.
1213
Bruckner, T.
1214
Zimmermann, H.; Schellnhuber, H.J.
1215
Svirezhev, Y.M.; von Bloh, W.; Schellnhuber, H.J.
1216
Leimbach, M.; Bruckner, T.
1217
Bateman, I.J.; Turner, R.K.; Klein, R.J.T.; Langford, I.H.
1218
Reusswig, F.
1219
Lahmer, W.; Müller-Wohlfeil, D.I.; Krysanova, V.; Becker, A.
1220
Sprinz, D.F.
1221
Sche

...

2058
Rahmstorf, S.
2059
Rahmstorf, S.; Alley, R.B.
2060
Rahmstorf, S.
2061
Solanki, S.; Ohmura, A.; Beer, J.; Fröhlich, C.; Latif, M.; Rahmstorf, S.; Schönwiese, C.; Neu, U.
2062
Rahmstorf, S.
2063
Rahmstorf, S.
2064
Rahmstorf, S.
2065
Klein, R.J.T.
2066
Klein, R.J.T.; Smith, J.B.
2067
Smith, J.B.; Klein, R.J.T.; Huq, S.
2068
Sterr, H.; Klein, R.J.T.; Reese, S.
2069
Runge, D.
2070
Dirmeyer, P.; Hoff, H.
2071
Hoff, H.(.a.; Kabat, P.; Bates, B.; Hellmuth, M.; Bullock, A.; van Schaik, H.; Connor, R.
2072
Pahl-Wostl, C.; Hoff, H.; Meybeck, M.; Sorooshian, S.
2073
Hoff, H.
2074
Hoff, H.; Jaeger, C.; Leveque, C.; Vorosmarty, C.
2075
Hunt, J.C.R.; Orr, A.; Cresswell, D.; Owinoh, A.
2076
Thonicke, K.; Venevsky, S.; Cramer, W.
2077
Thonicke, K.; Cramer, W.
2078
Thonicke, K.
2079
Alcamo, J.; Klein, R.J.T.; Carius, A.; Acosta-Michlik, L.; Krömker, K.; Tänzler, D.; Eierdanz, F.
2080
Füssel, H.M.; Klein, R.J.T.
2081
Tol, R.S.J.; Klein, R.J.T.; Nicholls, R.J.
2083
Klein, R.J.T.; Schipper, E.L.; Dess

2220
Scheffran, J.; Battaglini, A.; Weber, M.
2221
Becker, A.; Grünwald, U.
2222
Cossu, A.; Battaglini, A.; Bindi, M.
2223
Becker, A.; Bonell, M.; Feddes, R.A.; Krysanova, V.; McDonnell, J.; Schulze, R.E.; Valentin, C.
2224
Lotze-Campen, H.
2225
Reusswig, F.; Kropp, J.; Welp, M.
2226
Svirejeva-Hopkins, A.; Schellnhuber, H.J.; Pomaz, V.
2227
Jørgensen, S.E.; Svirezhev, Y.
2228
Svirezhev, Y.
2229
Stock, M.
2230
Stock, M.
2231
Stock, M.
2232
Stock, M.
2233
Stock, M.
2234
Stock, M.
2235
Stock, M.
2236
Bronstert, A.; Carrera, J.; Leavesley, G.; Mölders, N.
2237
Holzbecher, E.; Bonell, M.; Bronstert, A.; Vasiliev, O.
2238
Carrera, J.; Band, L.; Bronstert, A.; Kabat, P.; Mölders, N.
2239
Bronstert, A.; Holzbecher, E.; Kabat, P.; Vasiliev, O.
2240
Bronstert, A.; Mölders, N.
2241
Krol, M.; Bronstert, A.; Jaeger, A.
2242
Bronstert, A.
2243
Bronstert, A.; Engel, H.
2245
Krol, M.; Jaeger, A.; Bronstert, A.; Güntner, A.
2246
Bárdossy, A.; Bronsert, A.; Buiteveld, H.; Disse, M.; Fritsch, U.; Katzenm

2356
Haug, G.H.; Ganopolski, A.; Sigman, D.M.; Rosell-Mele, A.; Swann, G.E.A.; Tiedemann, R.; Jaccard, S.L.; Bollmann, J.; Maslin, M.A.; Leng, M.J.; Eglinton, G.
2357
Tuenter, E.; Weber, S.L.; Hilgen, F.J.; Lourens, L.J.; Ganopolski, A.
2358
Patt, A.G.; Klein, R.J.T.; de la Vega-Leinert, A.C.
2359
Nicholls, R.J.; Klein, R.J.T.; Salomons, W.
2360
Nunneri, C.; Turner, R.K.; Cieslak, A.; Kannen, A.; Klein, R.J.T.; Ledoux, L.; Marquenie, J.M.; Mee, L.D.; Moncheva, S.; Nicholls, R.J.; Salomons, W.; Sardá, R.; Stive, M.J.F.; Vellinga, T.
2361
Rochelle-Newall, E.; Klein, R.J.T.; Nicholls, R.J.; Barrett, K.; Behrendt, H.; Bresser, T.H.M.; Cieslak, A.; de Bruin, E.F.L.M.; Edwards, T.; Herman, P.M.J.; Laane, R.P.W.M.; Ledoux, L.; Lindeboom, H.; Lise, W.; Moncheva, S.; Moschella, P.S.; Stive, M.J.F.; Vermaat, J.E.
2362
Ionescu, C.; Klein, R.J.T.; Hinkel, J.; Kavi Kumar, K.S.; Klein, R.
2363
Klein, R.J.T.; Dougherty, W.W.; Alam, M.; Rahman, A.
2364
Acosta-Michlik, L.; Kavi Kumar, K.S.; Klein, R.J.

2518
Reusswig, F.
2519
Lantermann, E.D.; Reusswig, F.; Schuster, K.; Schwarzkopf, J.
2520
Grothmann, T.; Linneweber, V.; Reusswig, F.
2521
Runge, D.; Reusswig, F.
2522
Reusswig, F.
2523
Reusswig, F.
2524
Reusswig, F.; Kropp, J.; Welp, M.
2525
Reusswig, F.
2526
Reusswig, F.
2527
Reusswig, F.
2529
Reusswig, F.
2530
Reusswig, F.; Schwarzkopf, J.; Pohlenz, P.
2531
Reusswig, F.; Schwarzkopf, J.; Pohlenz, P.
2532
Reusswig, F.
2533
Reusswig, F.
2534
Stock, M.
2535
Doktor, D.; Badeck, F.W.; Hattermann, F.; Schaber, J.; McAllister, M.
2536
Hattermann, F.; Krysanova, V.; Habeck, A.
2537
Schellnhuber, H.J.
2538
Calov, R.; Greve, R.
2539
Schröter, D.; Cramer, W.; Leemans, R.; Prentice, C.; Araujo, M.B.; Arnell, N.W.; Bondeau, A.; Bugmann, H.; Carter, T.R.; Gracia, C.A.; de la Vega-Leinert, A.C.; Erhard, M.; Ewert, F.; Glendining, M.; House, J.I.; Kankaanpäa, S.; Klein, R.J.T.; Lavorel, S.; Lindner, M.; Metzger, M.J.; Meyer, J.; Mitchell, T.D.; Reginster, I.; Rounsevell, M.; Sabate, S.; Sitch, S.; 

KeyboardInterrupt: 

In [ ]:
shortenNames('editors')

In [9]:
authors = split('authors', ';', 'name')
findIllegalNames(authors, 'name', illegalCharsRegex)

90                                           Del‚colle, R.
1285                                  Gu‚rif, M. Delecolle
1294                                         Del‚colle, R.
1828                                         G‚rard, J.-C.
1957     IGBP Terrestrical Carbon Working Group (incl. ...
2351                 WASA Group (Carretero J.C.,  Gomes M.
3373                             Toth, F.L. (guest editor)
3773               et al. (including Schellnhuber,  H.-J.)
4215                     (in co-operation with Becker,  D.
4216                                      Ballerstedt, K.)
4404                                          ï¿½quist, M.
4462                                          Noguï¿½s, S.
5295                        Hoff, H. (contributing author)
7715                                          Kï¿½hler, J.
7809                                         FrÃ¶hlich, J.
7936                                        Klï¿½cking, B.
7937                                          Strï¿½bl, 

In [8]:
editors = split('editors', ';', 'name')
findIllegalNames(editors, 'name', illegalCharsRegex)

318                                            Sabat‚, S.
375                                           gren, G.I.
657     Ministerium für Landwirtschaft,  Umweltschutz ...
1017                                   ORiordan, T. (ed.)
Name: name, dtype: object

In [9]:
oldDepartmentNames = split('oldDepartmentNames', ',', 'name')
oldDepartmentNames['name'].unique()

array(['Library', 'Global Change & Natural Systems',
       'Integrated Systems Analysis', 'Climate Research',
       'Social Systems', 'Data & Computation', 'BAHC', 'Climate System'],
      dtype=object)

In [ ]:
df['type'].unique()

In [ ]:
df['publisher'].dropna().unique()

In [ ]:
df['journal'].dropna().unique()

In [ ]:
findIllegalNames(df, 'startpage', '[^0-9]').unique()

In [ ]:
findIllegalNames(df, 'endpage', '[^0-9]').unique()

In [ ]:
df['issue'].dropna().unique()
# findIllegalNames(df, 'issue', '[^0-9]').unique()

In [ ]:
df['vol'].dropna().unique()

In [ ]:
df['year'].dropna().unique()

In [ ]:
findIllegalNames(df, 'place', illegalCharsRegex)

In [ ]:
df['link'].dropna().unique()

# findIllegalNames(df, 'issue', '^$http.*').unique()

In [ ]:
df['comment'].dropna().unique()

In [ ]:
df['keywordsAndPeerReview'].dropna().unique()

In [ ]:
df['DOI'].dropna().unique()

In [ ]:
df.loc[df['journal'].dropna().index]['type'].unique()

In [ ]:
# booktitle appears not only in 'inbook' but also in other types of publication
df.loc[df['booktitle'].dropna().index]['type'].unique()

In [ ]:
# df[~df['relation (= Serie)'].isnull()]
df['Serie'].dropna().unique().shape

In [ ]:
df[~df['Serie'].isnull()]['type'].unique()

In [ ]:
df['conference'].dropna().unique().shape

In [ ]:
df.loc[df['conference'].dropna().index]['type'].unique()

In [ ]:
df.loc[df['DOI'].dropna().index]['type'].unique()

In [ ]:
df.loc[df['startpage'].dropna().index]['type'].unique()

In [49]:
df.to_csv(outputFile, encoding="utf8")